In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv

In [7]:
def get_data(url):
    page = requests.get(url, headers=header)
    soup = BeautifulSoup(page.content, "html.parser")
    return soup

In [8]:
def get_car_details(soup):

    titles = soup.find_all('h2', attrs={'class': "title"})
    car_names = [title.text for title in titles]
    
    stock_type = soup.find_all('p', attrs={'class': "stock-type"})
    usages = [usage.text for usage in stock_type]
    
    distance_run = soup.find_all(data_qa="mileage")
    mileages = [mileage.text for mileage in distance_run]
    
    primary_price = soup.find_all('span', attrs={'class': "primary-price"})
    prices = [price.text for price in primary_price]
    
    website_link = soup.find_all('a', attrs={'class': "vehicle-card-link js-gallery-click-link"})
    Web = "https://www.cars.com"
    links = [Web + link.get("href") for link in website_link]
    
    dealers = soup.find_all('div', attrs={'class': "dealer-name"})
    dealer_names = [dealer.text.strip() for dealer in dealers]
    
    distance = soup.find_all('div', attrs={'class': "miles-from"})
    miles_from = [mile_from.text.strip() for mile_from in distance]
         
    count = len(car_names)
    used_car_count = 0
    for car_name, usage, price, link, dealer_name, mile_from in zip(car_names, usages, prices, links, dealer_names, miles_from):
        if usage == 'New':
            new_row = {'Name': car_name, 'Stock_type': usage, 'Mileage': 0, 'Primary_price': price, 'Website_link': link, 'Dealer_name': dealer_name, 'Distance_from': mile_from}    
        else:
            new_row = {'Name': car_name, 'Stock_type': usage, 'Mileage': mileages[used_car_count], 'Primary_price': price, 'Website_link': link, 'Dealer_name': dealer_name, 'Distance_from': mile_from}
            used_car_count += 1
        df.loc[len(df)] = new_row
        
    return df


In [9]:
def get_next_page(soup):
    search_page = soup.find('a', attrs={'aria-label': "Next page"}).get('href')
    return search_page


In [10]:
website = "https://www.cars.com/"
search_page = "shopping/results/?body_style_slugs[]=sedan&dealer_id=&keyword=&list_price_max=&list_price_min=&makes[]=&maximum_distance=30&mileage_max=&monthly_payment=&page_size=20&sort=best_match_desc&year_max=&year_min=&zip=02466"
header = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})       #user agent


In [11]:
df_header = ['Name', 'Stock_type', 'Mileage', 'Primary_price', 'Website_link', 'Dealer_name', 'Distance_from']
df = pd.DataFrame(columns = df_header)

with open('CarsInventory.csv', 'w', encoding='utf8',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(df_header)

In [14]:
#while True:
#    if not search_page:
#        break

for i in range(1,4):
    soup = get_data(website+search_page)
    df_car_details = get_car_details(soup)
    search_page = get_next_page(soup)

df_car_details.to_csv(r'C:\Users\nthkr\OneDrive\Desktop\Work\Projects\PortfolioProjects\Web scraping\CarsInventory.csv', mode='a', index=False, header=False)

    